connect with google drive

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np

# Upload file "sales.csv"
uploaded = files.upload()
sales = pd.read_csv('sales.csv')

# Upload file "prices.csv"
uploaded = files.upload()
prices = pd.read_csv('prices.csv')

Saving sales.csv to sales.csv


Saving prices.csv to prices.csv


connect with google drive

In [ ]:
print("Sales data:")
print(sales)
print("Prices data:")
print(prices)

Sales data:
     product_id     ordered_at  quantity_ordered
0       3998909  9/18/18 17:51                 1
1       3998909  9/18/18 12:52                 1
2       3998909  9/18/18 11:33                 1
3       3998909  9/18/18 18:47                 1
4       3998909  9/18/18 17:36                 1
..          ...            ...               ...
170     4085861  9/13/18 23:29                 1
171          64  9/11/18 19:32                 1
172          64  9/14/18 20:10                 1
173          64  9/12/18 16:00                 1
174          64  9/12/18 13:23                 1

[175 rows x 3 columns]
Prices data:
    product_id  old_price  new_price     updated_at
0           64     270000     239000  9/10/18 16:37
1      3954203      60000      64000  9/11/18 11:54
2      3954203      60500      57500  9/17/18 22:59
3      3954203      64000      60500   9/15/18 3:49
4      3954203      68800      60000  9/10/18 16:32
5      3998909      15500      16500   9/16/18 5:09

In [ ]:
# Check null in sales
sales.isna().sum()

product_id          0
ordered_at          0
quantity_ordered    0
dtype: int64

In [ ]:
# Check null in prices
prices.isna().sum()

product_id    0
old_price     0
new_price     0
updated_at    0
dtype: int64

Calculating the total revenue of each product at each price

In [ ]:
#convert ordered_at, updated_at,to datetime object

sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
prices['updated_at'] = pd.to_datetime(prices['updated_at'])

<ipython-input-5-d55fb1b2133a>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
<ipython-input-5-d55fb1b2133a>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  prices['updated_at'] = pd.to_datetime(prices['updated_at'])


In [ ]:
sales = sales.sort_values(by='ordered_at')
sales

,product_id,ordered_at,quantity_ordered
86,3954203,2018-09-11 01:43:00,1
28,4085861,2018-09-11 06:26:00,1
26,4085861,2018-09-11 06:53:00,1
27,4085861,2018-09-11 08:24:00,1
123,4085861,2018-09-11 09:30:00,1
...,...,...,...
67,4085861,2018-09-18 20:23:00,1
77,4085861,2018-09-18 20:43:00,1
79,4085861,2018-09-18 20:54:00,1
87,3954203,2018-09-18 21:26:00,1


In [ ]:
prices = prices.sort_values(by='updated_at')
prices

,product_id,old_price,new_price,updated_at
4,3954203,68800,60000,2018-09-10 16:32:00
7,3998909,19000,17000,2018-09-10 16:35:00
0,64,270000,239000,2018-09-10 16:37:00
11,4085861,60000,53500,2018-09-11 08:51:00
1,3954203,60000,64000,2018-09-11 11:54:00
9,4085861,53500,67000,2018-09-12 03:51:00
6,3998909,17000,15500,2018-09-13 06:43:00
13,4085861,67000,62500,2018-09-13 06:43:00
3,3954203,64000,60500,2018-09-15 03:49:00
12,4085861,62500,58000,2018-09-15 03:51:00


In [ ]:
# use merge_asof with direction='nearest' to merge sales and prices
merge_nearest = pd.merge_asof(sales, prices, left_on='ordered_at', right_on='updated_at', by='product_id', direction='nearest')

In [ ]:
# compare ordered_at and "nearest" updated_at of each order to find the correct price

merge_nearest['final_price'] = np.where(merge_nearest['ordered_at'] >= merge_nearest['updated_at'],
                                         merge_nearest['new_price'], merge_nearest['old_price'])

In [ ]:
merge_nearest['revenue'] = merge_nearest['quantity_ordered'] * merge_nearest['final_price']
total_revenue = merge_nearest.groupby('product_id', as_index=False)['revenue'].sum()
total_revenue

,product_id,revenue
0,64,956000
1,3954203,877500
2,3998909,280500
3,4085861,8247500


In [ ]:
merge_nearest['revenue'] = merge_nearest['quantity_ordered'] * merge_nearest['final_price']
revenue_by_product_and_price = merge_nearest.groupby(['product_id', 'final_price'], as_index=False)['revenue'].sum()
revenue_by_product_and_price

,product_id,final_price,revenue
0,64,239000,956000
1,3954203,57500,57500
2,3954203,60000,180000
3,3954203,64000,640000
4,3998909,15500,15500
5,3998909,16500,231000
6,3998909,17000,34000
7,4085861,52000,1040000
8,4085861,53500,2140000
9,4085861,58000,2204000


In [ ]:
# use merge_asof to find new price for each item ordered

merge_backward = pd.merge_asof(sales, prices, left_on='ordered_at', right_on='updated_at', by='product_id')
merge_backward.head()

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at
0,3954203,2018-09-11 01:43:00,1,68800.0,60000.0,2018-09-10 16:32:00
1,4085861,2018-09-11 06:26:00,1,NaN,NaN,NaT
2,4085861,2018-09-11 06:53:00,1,NaN,NaN,NaT
3,4085861,2018-09-11 08:24:00,1,NaN,NaN,NaT
4,4085861,2018-09-11 09:30:00,1,60000.0,53500.0,2018-09-11 08:51:00


In [ ]:
# fill null values for the orders made before the first update of each product
# after filling, new_price column is the price we need
merge_backward['final_price'] = np.where(merge_backward['new_price'].isnull(), merge_backward['old_price'], merge_backward['new_price'])
merge_backward['revenue'] = merge_backward['quantity_ordered'] * merge_backward['final_price']
merge_backward.head()


,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,revenue,final_price
0,3954203,2018-09-11 01:43:00,1,68800.0,60000.0,2018-09-10 16:32:00,60000.0,60000.0
1,4085861,2018-09-11 06:26:00,1,60000.0,60000.0,NaT,60000.0,60000.0
2,4085861,2018-09-11 06:53:00,1,60000.0,60000.0,NaT,60000.0,60000.0
3,4085861,2018-09-11 08:24:00,1,60000.0,60000.0,NaT,60000.0,60000.0
4,4085861,2018-09-11 09:30:00,1,60000.0,53500.0,2018-09-11 08:51:00,53500.0,53500.0


In [ ]:
revenue_by_product_and_price = merge_backward.groupby(['product_id', 'final_price'], as_index=False)['revenue'].sum()
revenue_by_product_and_price

,product_id,final_price,revenue
0,64,239000.0,956000.0
1,3954203,57500.0,57500.0
2,3954203,60000.0,180000.0
3,3954203,64000.0,640000.0
4,3998909,15500.0,15500.0
5,3998909,16500.0,231000.0
6,3998909,17000.0,34000.0
7,4085861,52000.0,1040000.0
8,4085861,53500.0,2140000.0
9,4085861,58000.0,2204000.0


In [ ]:
revenue_by_product = merge_backward.groupby('product_id', as_index=False)['revenue'].sum()
revenue_by_product

,product_id,revenue
0,64,956000.0
1,3954203,877500.0
2,3998909,280500.0
3,4085861,8247500.0
